In [3]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd

def get_db_connection():
    server = SSHTunnelForwarder(
        ('letsrise.myonline.works', 22),  # SSH server and port
        ssh_username='ubuntu',  # SSH username
        ssh_pkey='~/.ssh/id_letsrise',  # Path to your private key
        remote_bind_address=('localhost', 5432)  # Database server and port
    )

    server.start()

    conn = psycopg2.connect(
        dbname='letsrise_v1',
        user='letsrise_intern',
        password='letsrise',
        host='localhost',
        port=server.local_bind_port  # Use the dynamically assigned local port
    )
    return conn, server

In [21]:
conn, server = get_db_connection()
query1 = """
SELECT * FROM public.user_info
ORDER BY user_id ASC 
"""
df = pd.read_sql(query1, conn)
conn.close()
server.stop()

In [22]:
df.head()
df_exploded = df.assign(industry_experience=df['industry_experience'].str.split('\n')).explode('industry_experience')
df_exploded.head()

,user_id,name,email,age,linkedin_url,education_level,employment_status,entrepreneurial_experience,current_startup_stage,number_of_startups,role_in_entrepreneurship,industry_experience,number_of_previous_startups,location,gender,startup_name
0,02e6fb77-58f9-4930-9ee3-47740bfe618c,Shamimuzzaman Chowdhury,shamim.zaman.chowdhury@gmail.com,39,www.linkedin.com/in/shamim-chowdhury-cfa-8019414,Bachelors,Self Employed,3.0,Revenue Stage,1.0,None,None,0.0,Abu Dhabi United Arab Emirates,Male,Letsrise
1,038d31e5-aab7-40c3-bd28-868aa064a504,Dev Kalavadiya,dk3936@nyu.edu,23,https://www.linkedin.com/in/dev-k-a80561130/,Bachelors,Student,0.0,Validation Stage,2.0,CTO - Technology,Education,0.0,Abu Dhabi United Arab Emirates,Male,HoloLearn
1,038d31e5-aab7-40c3-bd28-868aa064a504,Dev Kalavadiya,dk3936@nyu.edu,23,https://www.linkedin.com/in/dev-k-a80561130/,Bachelors,Student,0.0,Validation Stage,2.0,CTO - Technology,Deep Technology,0.0,Abu Dhabi United Arab Emirates,Male,HoloLearn
1,038d31e5-aab7-40c3-bd28-868aa064a504,Dev Kalavadiya,dk3936@nyu.edu,23,https://www.linkedin.com/in/dev-k-a80561130/,Bachelors,Student,0.0,Validation Stage,2.0,CTO - Technology,Engineering,0.0,Abu Dhabi United Arab Emirates,Male,HoloLearn
1,038d31e5-aab7-40c3-bd28-868aa064a504,Dev Kalavadiya,dk3936@nyu.edu,23,https://www.linkedin.com/in/dev-k-a80561130/,Bachelors,Student,0.0,Validation Stage,2.0,CTO - Technology,Machine Learning,0.0,Abu Dhabi United Arab Emirates,Male,HoloLearn


In [25]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, MATCH
import pandas as pd

app = dash.Dash(__name__)

# Layout of the Dash app with the given color palette
app.layout = html.Div(style={'backgroundColor': '#f7f7f7', 'fontFamily': 'Poppins, sans-serif'}, children=[
    dcc.Dropdown(
        id='user-dropdown',
        options=[{'label': name, 'value': user_id} for user_id, name in zip(df['user_id'].unique(), df['name'].unique())],
        placeholder="Select a User",
        style={'marginBottom': '10px'}
    ),
    dcc.Dropdown(
        id='type-dropdown',
        options=[
            {'label': 'Business Partner', 'value': 'business_partner'},
            {'label': 'Mentor', 'value': 'mentor'}
        ],
        placeholder="Select Type",
        style={'marginBottom': '10px'}
    ),
    dcc.Dropdown(
        id='industry-dropdown',
        placeholder="Select Industry Experience",
        style={'marginBottom': '20px'}
    ),
    html.Div(id='results', style={'color': '#111', 'padding': '20px', 'border': '1px solid #ccc', 'borderRadius': '5px', 'boxShadow': '0 1px 3px rgba(0, 0, 0, 0.1)', 'backgroundColor': '#fff'}),
])

# Callback to update the industry dropdown based on the unique values in the DataFrame
@app.callback(
    Output('industry-dropdown', 'options'),
    Input('type-dropdown', 'value')
)
def set_industry_options(selected_type):
    if selected_type:
        unique_industries = df_exploded['industry_experience'].dropna().unique()
        return [{'label': industry, 'value': industry} for industry in unique_industries]
    return []

# Callback to update the results based on selected user, type, and industry experience
@app.callback(
    Output('results', 'children'),
    [Input('user-dropdown', 'value'),
     Input('type-dropdown', 'value'),
     Input('industry-dropdown', 'value')]
)
def update_results(selected_user, selected_type, selected_industry):
    if not selected_user or not selected_type or not selected_industry:
        return "Please make all selections."

    filtered_df = df_exploded[df_exploded['industry_experience'] == selected_industry]
    
    if selected_type == 'business_partner':
        top_5 = filtered_df[filtered_df['user_id'] != selected_user].head(5)
    elif selected_type == 'mentor':
        top_5 = filtered_df[filtered_df['user_id'] != selected_user].sort_values(by='entrepreneurial_experience', ascending=False).head(5)
    
    return html.Div([
        html.Div([
            html.Div([
                html.Div([
                    html.P(f"{row['name']} - {row['industry_experience']} - {row['entrepreneurial_experience']} years of experience"),
                    html.Button('Introduce', id={'type': 'introduce-button', 'index': idx}, n_clicks=0, style={'float': 'right'})
                ], style={'display': 'flex', 'justifyContent': 'space-between', 'alignItems': 'center'}),
                html.Div(id={'type': 'email-form', 'index': idx}, style={'display': 'none'})
            ])
        ]) for idx, row in top_5.iterrows()
    ])

# Callback to show the email form when the button is clicked
@app.callback(
    Output({'type': 'email-form', 'index': MATCH}, 'style'),
    Input({'type': 'introduce-button', 'index': MATCH}, 'n_clicks'),
    State({'type': 'email-form', 'index': MATCH}, 'style'),
    prevent_initial_call=True
)
def show_email_form(n_clicks, current_style):
    if n_clicks > 0:
        return {'display': 'block', 'paddingTop': '10px'}
    return current_style

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


In [27]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from email.mime.text import MIMEText
import base64

# Path to your service account key file for Gmail API
key_file_path = 'letsrise-caed029c5496.json'

# Create credentials
credentials = service_account.Credentials.from_service_account_file(
    key_file_path, scopes=['https://www.googleapis.com/auth/gmail.send'])

# If you're impersonating a user, specify the user_email
user_email = 'svn9705@nyu.edu'
credentials = credentials.with_subject(user_email)

try:
    # Build the Gmail service
    service = build('gmail', 'v1', credentials=credentials)
    print("Successfully authenticated with Gmail API.")

    # Example: Sending an email
    to_email = 'svn9705@nyu.edu'
    subject = 'Test Email'
    message_text = 'This is a test email sent from Gmail API.'

    message = MIMEText(message_text)
    message['to'] = to_email
    message['from'] = 'svn9705@nyu.edu'
    message['subject'] = subject
    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

    # Send the email
    sent_message = service.users().messages().send(userId='me', body={'raw': raw_message}).execute()
    print('Message Id: %s' % sent_message['id'])

except Exception as e:
    print(f"Error occurred: {e}")


Successfully authenticated with Gmail API.
Error occurred: ('unauthorized_client: Client is unauthorized to retrieve access tokens using this method, or client not authorized for any of the scopes requested.', {'error': 'unauthorized_client', 'error_description': 'Client is unauthorized to retrieve access tokens using this method, or client not authorized for any of the scopes requested.'})
